In [ ]:
#TODO: CHANGE THIS BASED ON YOUR OWN LOCAL SETTINGS
#MY_HOME_ABS_PATH = "/Users/jetcalz07/Desktop/MIDS/W210_Capstone/co2-flux-hourly-gpp-modeling"
MY_HOME_ABS_PATH = "/root/co2-flux-hourly-gpp-modeling/"

In [ ]:
from tqdm import tqdm
from tempfile import TemporaryFile
import pandas as pd
import csv
import os
os.chdir(MY_HOME_ABS_PATH)
import sys
sys.path.append(os.path.abspath("./code/src/tools"))
from CloudIO.AzStorageClient import AzStorageClient

In [ ]:
root_dir =  MY_HOME_ABS_PATH
tmp_dir =  root_dir + os.sep + '.tmp'
cred_dir = root_dir + os.sep + '.cred'
az_cred_file = cred_dir + os.sep + 'azblobcred.json'

if not (os.path.exists(tmp_dir)):
    print("Making tmp_dir")
    os.mkdir(tmp_dir)

In [ ]:
# Initialize Azure Storage Client
azStorageClient = AzStorageClient(az_cred_file)
container_name = 'full-half-hourly-raw'
blob_name_list = azStorageClient.listBlobs(container_name)

In [ ]:
# List blobs names under the container "data_full_half_hourly"
for i, blob in enumerate(blob_name_list):
    print(f'{i:3} {blob.name}')
    if i > 4:
        break

In [ ]:
# Check one example to get colnames
ex_file = tmp_dir + os.sep + 'site_example_ZM-Mon.csv' 
site_ex = pd.read_csv(ex_file)
col_names = site_ex.columns
site_ex.head()

In [ ]:
import shutil

In [ ]:
# Download all HH site file to .tmp dir
for i, blob in tqdm(enumerate(blob_name_list)):
    blob_name = blob.name
    data = azStorageClient.downloadBlob(container_name, blob_name)
    
    # Create a temporary file to write the CSV file with column names
    with TemporaryFile(mode='w+', newline='') as tmp_file:
        writer = csv.writer(tmp_file)
        
        # Write the column names as the first row
        writer.writerow(col_names)
        
        # Write the data as the remaining rows
        writer.writerows(csv.reader(data.decode().splitlines()))
        
        # Save the temporary file to disk
        filename = os.path.join(tmp_dir, blob_name)
        with open(filename, "w", newline='') as f:
            tmp_file.seek(0)
            f.write(tmp_file.read())

In [ ]:
# Check that the Azure df == the locally uploaded example from GDrive
local_filename = tmp_dir + os.sep + f'data_full_half_hourly_raw_v0_1_ZM-Mon.csv' 
site_df = pd.read_csv(local_filename)
pd.equals(site_df, site_ex)

In [ ]:
tmp_dir_size = len([x for x in os.listdir(tmp_dir) if 'data_full_half_hourly_raw' in x])
print(f"Number of site files in tmp_dir: {tmp_dir_size}")